In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold

import os
import sys
sys.path.append('..')

from utils import calc_wap, calc_wap2, log_return, realized_volatility, count_unique, calc_mean_importance, calc_model_importance, plot_importance, rmspe, feval_RMSPE
from preprocess import create_all_feature

In [2]:
NB = "009"
feature_path = "../output/feature/nb" + NB
model_path = "../output/model/nb" + NB
if not os.path.isdir(feature_path):
    os.makedirs(feature_path)
if not os.path.isdir(model_path):
    os.makedirs(model_path)


In [3]:
data_dir = '../input/optiver-realized-volatility-prediction/'

In [4]:
df_train, df_test = create_all_feature()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  8.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [5]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_price1_nunique,ask_price1_nunique,bid_price2_nunique,ask_price2_nunique,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_price1_nunique_450,ask_price1_nunique_450,bid_price2_nunique_450,ask_price2_nunique_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_price1_nunique_300,ask_price1_nunique_300,bid_price2_nunique_300,ask_price2_nunique_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_price1_nunique_150,ask_price1_nunique_150,bid_price2_nunique_150,ask_price2_nunique_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id,stock_id_target_enc,log_return_realized_volatility_mean_stock,log_return_realized_volatility_std_stock,log_return_realized_volatility_max_stock,log_return_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return_realized_volatility_450_mean_stock,log_return_realized_volatility_450_std_stock,log_return_realized_volatility_450_max_stock,log_return_realized_volatility_450_min_stock,log_return2_realized_volatility_450_mean_stock,log_return2_realized_volatility_450_std_stock,log_return2_realized_volatility_450_max_stock,log_return2_realized_volatility_450_min_stock,log_return_realized_volatility_300_mean_stock,log_return_realized_volatility_300_std_stock,l

In [6]:
df_test.head()

,stock_id,time_id,row_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_price1_nunique,ask_price1_nunique,bid_price2_nunique,ask_price2_nunique,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_price1_nunique_450,ask_price1_nunique_450,bid_price2_nunique_450,ask_price2_nunique_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_price1_nunique_300,ask_price1_nunique_300,bid_price2_nunique_300,ask_price2_nunique_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_price1_nunique_150,ask_price1_nunique_150,bid_price2_nunique_150,ask_price2_nunique_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id_target_enc,log_return_realized_volatility_mean_stock,log_return_realized_volatility_std_stock,log_return_realized_volatility_max_stock,log_return_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return_realized_volatility_450_mean_stock,log_return_realized_volatility_450_std_stock,log_return_realized_volatility_450_max_stock,log_return_realized_volatility_450_min_stock,log_return2_realized_volatility_450_mean_stock,log_return2_realized_volatility_450_std_stock,log_return2_realized_volatility_450_max_stock,log_return2_realized_volatility_450_min_stock,log_return_realized_volatility_300_mean_stock,log_return_realized_volatility_300_std_stock,

# LightGBM

In [7]:
X = df_train.drop(['row_id','target'],axis=1)
y = np.log(df_train['target'])

In [8]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8, # 利用する特徴量の割合
      'bagging_fraction': 0.8,
  }

In [9]:
kf = KFold(n_splits=5, random_state=55, shuffle=True)
oof = pd.DataFrame()
models = []
scores = 0.0

In [10]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'Fold: {fold+1}')
    X_train, y_train = X.loc[trn_idx], y.loc[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y.loc[val_idx]
    
    # RMSPEで最適化を行うため? RMSEの損失関数に1/yi^2の重み付けをすればOK
    # https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/250324
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight=weights)
    
    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference=lgbm_train, weight=weights)
    
    model = lgbm.train(params=params,
                  train_set=lgbm_train,
                  valid_sets=[lgbm_train, lgbm_valid],
                  num_boost_round=5000,         
                  feval=feval_RMSPE,
                  verbose_eval=100,
                  categorical_feature = ['stock_id']                
                 )
    
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)

Fold: 1


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.250224 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 54682
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 246


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score -5.611131
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.339619	training's RMSPE: 0.06011	valid_1's rmse: 0.340383	valid_1's RMSPE: 0.06024
[200]	training's rmse: 0.258134	training's RMSPE: 0.04569	valid_1's rmse: 0.259515	valid_1's RMSPE: 0.04593
[300]	training's rmse: 0.238132	training's RMSPE: 0.04215	valid_1's rmse: 0.239854	valid_1's RMSPE: 0.04245
[400]	training's rmse: 0.230257	training's RMSPE: 0.04075	valid_1's rmse: 0.232206	valid_1's RMSPE: 0.04109
[500]	training's rmse: 0.225533	training's RMSPE: 0.03992	valid_1's rmse: 0.227664	valid_1's RMSPE: 0.04029
[600]	training's rmse: 0.222265	training's RMSPE: 0.03934	valid_1's rmse: 0.224594	valid_1's RMSPE: 0.03975
[700]	training's rmse: 0.219445	training's RMSPE: 0.03884	valid_1's rmse: 0.221927	valid_1's RMSPE: 0.03928
[800]	training's rmse: 0.216987	training's RMSPE: 0.03841	valid_1's rmse: 0.219594	valid_1's RMSPE: 0.03886
[900]	training's rmse

...

[LightGBM] [Info] Start training from score -5.612170
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.339678	training's RMSPE: 0.06011	valid_1's rmse: 0.339384	valid_1's RMSPE: 0.06011
[200]	training's rmse: 0.258228	training's RMSPE: 0.0457	valid_1's rmse: 0.258516	valid_1's RMSPE: 0.04578
[300]	training's rmse: 0.238222	training's RMSPE: 0.04216	valid_1's rmse: 0.23901	valid_1's RMSPE: 0.04233
[400]	training's rmse: 0.230595	training's RMSPE: 0.04081	valid_1's rmse: 0.231675	valid_1's RMSPE: 0.04103
[500]	training's rmse: 0.225667	training's RMSPE: 0.03994	valid_1's rmse: 0.226918	valid_1's RMSPE: 0.04019
[600]	training's rmse: 0.222278	training's RMSPE: 0.03934	valid_1's rmse: 0.223662	valid_1's RMSPE: 0.03961
[700]	training's rmse: 0.219524	training's RMSPE: 0.03885	valid_1's rmse: 0.221079	valid_1's RMSPE: 0.03915
[800]	training's rmse: 0.21705	training's RMSPE: 0.03841	valid_1's rmse: 0.218744	valid_1's RMSPE: 0.03874
[900]	training's rmse: 0

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.243027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 54675
[LightGBM] [Info] Number of data points in the train set: 343146, number of used features: 246


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score -5.610873
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.339621	training's RMSPE: 0.06012	valid_1's rmse: 0.339694	valid_1's RMSPE: 0.0601
[200]	training's rmse: 0.258236	training's RMSPE: 0.04571	valid_1's rmse: 0.258499	valid_1's RMSPE: 0.04574
[300]	training's rmse: 0.238242	training's RMSPE: 0.04217	valid_1's rmse: 0.238849	valid_1's RMSPE: 0.04226
[400]	training's rmse: 0.230357	training's RMSPE: 0.04077	valid_1's rmse: 0.231232	valid_1's RMSPE: 0.04091
[500]	training's rmse: 0.225399	training's RMSPE: 0.0399	valid_1's rmse: 0.226451	valid_1's RMSPE: 0.04007
[600]	training's rmse: 0.221991	training's RMSPE: 0.03929	valid_1's rmse: 0.223231	valid_1's RMSPE: 0.0395
[700]	training's rmse: 0.219387	training's RMSPE: 0.03883	valid_1's rmse: 0.220817	valid_1's RMSPE: 0.03907
[800]	training's rmse: 0.217116	training's RMSPE: 0.03843	valid_1's rmse: 0.218744	valid_1's RMSPE: 0.0387
[900]	training's rmse: 0.

In [11]:
scores

0.0352

# Model保存

In [12]:
import pickle

for i, model in enumerate(models):
    pickle.dump(model, open(model_path+"/lgbm"+str(i)+".pkl", 'wb'))

# 特徴量保存

In [13]:
pickle.dump(df_train, open(os.path.join(feature_path, "train.pkl"), 'wb'))
pickle.dump(df_test, open(os.path.join(feature_path, "test.pkl"), 'wb'))

In [14]:
X_valid.columns.to_series().to_csv(os.path.join(feature_path, "columns.csv"), index=False)

In [15]:
X_valid.columns

Index(['log_return_realized_volatility', 'log_return_mean', 'log_return_std',
       'log_return_sum', 'log_return2_realized_volatility', 'log_return2_mean',
       'log_return2_std', 'log_return2_sum', 'wap_balance_mean',
       'wap_balance_std',
       ...
       'trade_log_return_realized_volatility_450_max_time',
       'trade_log_return_realized_volatility_450_min_time',
       'trade_log_return_realized_volatility_300_mean_time',
       'trade_log_return_realized_volatility_300_std_time',
       'trade_log_return_realized_volatility_300_max_time',
       'trade_log_return_realized_volatility_300_min_time',
       'trade_log_return_realized_volatility_150_mean_time',
       'trade_log_return_realized_volatility_150_std_time',
       'trade_log_return_realized_volatility_150_max_time',
       'trade_log_return_realized_volatility_150_min_time'],
      dtype='object', length=246)